In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = cpu_count()*5
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [ ]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [ ]:
model = TypePredictor()

model.fit(x_train, y_train)

In [ ]:
y_pred = model._model.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

In [ ]:
mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_pred[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_test[mask])

for true, pred in zip(true_labels, predicted_labels):
    print(true, pred)